# What is Modem?

I would bet that you've heard of a modem.  But do you know what it is?  A modem is shorthand for *modulator-demodulation*.

Modulation is all about transmitting information through a medium, or *channel*, in hopefully some optimal way.  Information is
conveyed by modifying a known signal in a prescribed way that is understood by a receiver.  Here is the type of signal that we are interested in adding information to:  

\begin{equation}
    x(t) = \alpha (t) \cos \left(\omega (t) t + \phi (t) \right). 
\end{equation}

It's just your standard sinusoidal signal with information-bearing parameters: amplitude $\alpha(t)$, angular frequency $\omega (t)$, and phase $\phi(t)$.  


## Binary Phase Shift Keying

The standard derivation of PSK modulation follows from standard trigonometric identities.  

\begin{align}
    x_{c}(t) &= \cos (2\pi f_{c} t + \phi(t)) \\
             &= \cos (\phi(t)) \cos (2\pi f_{c} t) - \sin (\phi(t)) \sin (2\pi f_{c} t) \\
             &= I(t) \cos (2\pi f_{c} t) - Q(t) \sin (2\pi f_{c} t)
\end{align}

The $I(t)$ and $Q(t)$ signals are commonly referred to as the inphase and quadrature signals, respectively.  In Binary Phase Shift Keying (BPSK), the $\phi(t)$ can only take on the values $0$ and $\pi$.  If you go through the math, this means that 
$Q(t) = 0$ for any $t$ and $I(t) = \pm 1$.  

At this point it's worth reiterating that we are trying to motivate digital communications and therefore $I(t)$ and $Q(t)$ are 
discrete signals.  That is we only have access to the two signals at a discrete set of points defined by an underlying sample time $t_{s}$.  

## Differential Encoding

Before we discuss the algorithmic details of differential encoding/decoding, it's important to motivate the reason for why it might be considered.

### Motivation

### The Algorithm
At the bit-level, the algorithm is extremely simple.  Given a sequence of input bits, ${ x_{i} }$ the differentially encoded stream of bits is defined by the following recurrence relation:

\begin{equation}
    y_{i} = y_{i-1} \oplus x_{i}
\end{equation}

The differential decoder runs in the receiver and recovers the original sequence from the encoded sequence as follows:

\begin{equation}
    z_{i} = y_{i-1} \oplus y_{i}
\end{equation}

So, why does this work?  Let's prove to ourselves that differentially decoding a differentially encoded bit sequence returns
the original sequence.  For each $i$,

\begin{align}
    z_{i} &= y_{i-1} \oplus y_{i} \\
          &= y_{i-1} \oplus \left( y_{i-1} \oplus x_{i} \right) \\
          &= x_{i} \oplus \left( y_{i-1} \oplus y_{i-1} \right) \\
          &= x_{i} \oplus 0 \\
          &= x_{i}
\end{align}

Of course a real communications channel will cause some of the $y_{i}$ bits to flip.  To see the channel's impact, let

\begin{equation}
    \bar{y}_{i} = y_{i} \oplus 1.
\end{equation}

and take a look at what happens when two consecutive bit errors occur:

\begin{align}
    z_{i} &= \bar{y}_{i-1} \oplus \bar{y}_{i} \\
          &= \left( y_{i-1} \oplus 1 \right) \oplus \left( y_{i} \oplus 1 \right) \\
          &= \left( y_{i-1} \oplus y_{i} \right) \oplus \left( 1 \oplus 1 \right) \\
          &= x_{i} \oplus 0 \\
          &= x_{i}. 
\end{align}

In other words, the differential decoder fixes two consecutive bit errors.





### Design Patterns
There are a variety of ways to implement a differential encoder/decoder.  I'm really after designing my algorithms in efficient and elegant ways.  One thing I want to try to avoid is being tied to a particular type of input.  I don't want to implement separate differential encoders when the bits are coming in one at a time and when they are given to me all at once.  I want to capture the *essence* of the algorithm in a flexible and reusable module.  And if I can do this using functional programming techniques, then that's just icing on the cake.  


The first thing I need is some way to maintain state during the course of the algorithm.  Here's one option.

In [4]:
class Register(object):
    def __init__(self,val=None):
        self.val = val
    def push(self, new_value):
        self.val = new_value
    def peek(self):
        return self.val

Okay, now let's try to capture the essence of a differential encoder.  We are going to use a concept from functional programming called *currying* to do it.  All this really means is taking advantage of the mathematical fact that a function of two arguments can be interpreted as a function of one argument that returns another function of one argument.  What does currying look like in Python? Let's look at a simple example before getting to the differential encoder.  Consider a simple `adder` function that takes two numbers and adds them together:

In [22]:
def adder(x,y):
    return x + y

Here's the curried version of this function.  

In [23]:
def adder_c(x):
    def inner(y):
        return x + y
    return inner

In [16]:
def diff_enc():
    state = Register(0)
    def step(bit, signal=None):
        if signal == 1:
            state.push(0)
            return bit
        curr   = state.peek()
        output = curr ^ bit
        state.push(output)
        return output
    
    return step

In [17]:
def diff_dec():
    state = Register(0)
    def step(bit, signal=None):
        if signal == 1:
            state.push(0)
            return bit
        curr   = state.peek()
        output = curr ^ bit
        state.push(bit)
        return output
    return step

In [18]:
enc = diff_enc()
dec = diff_dec()

In [19]:
bits_in = list(map(enc,[0,1,1,0,0,1,1,0]))

In [20]:
bits_out = list(map(dec,bits_in))
bits_out

[0, 1, 1, 0, 0, 1, 1, 0]

In [21]:
foo()

NameError: name 'foo' is not defined

In [62]:
list(map(foo,[1,1,0,1,0,1,1,0,1,0,0]))

[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]

```rust
fn diff_enc() -> Box<FnMut(i8) -> i8> {
    let mut reg: i8 = 0;
    Box :: new (move |x:i8| {
        reg ^= x;
        reg
    }
}
```

```c++
auto diff_encode = [] () {
    auto reg = std::make_shared<int>(0);
    auto step = [reg](int bit,int flip) {
        int out = *reg ^ bit;
        *reg = out ^ flip;
        return out;
    };
    return step;
};
```